In [133]:
import base64
from typing import Any
import math
import random
from sympy.ntheory import pollard_rho


In [134]:
# coder to copypaste
class Coder:
    @staticmethod
    def encode(text: str) -> list[int]:
        return [ord(i) for i in text]

    @staticmethod
    def decode(encoded_text: list[int]) -> str:
        return "".join([chr(i) for i in encoded_text])
    
    @staticmethod
    def encode_blocks(blocks: list[int]) -> Any:
        number_sequence = (' '.join(str(number) for number in blocks)).encode()
        encoded_data = base64.b64encode(number_sequence)
        return encoded_data.decode('ascii')
    
    @staticmethod
    def decode_blocks(cipher: Any) -> list[int]:
        number_sequence = base64.b64decode(cipher)

        decoded_numbers = []
        
        for str_number in number_sequence.split():
            decoded_numbers.append(int(str_number))
        
        return decoded_numbers

In [135]:
%run rsa_coder.ipynb

In [136]:
def generate_prime(bits=512):
    """Генерация простого числа заданной битности"""
    while True:
        p = random.getrandbits(bits)
        # Убедимся, что число нечетное и достаточно большое
        p |= (1 << bits - 1) | 1
        if pollard_rho(p) is None:
            return p


In [137]:
def extended_gcd(a, b):
    """Расширенный алгоритм Евклида"""
    if a == 0:
        return b, 0, 1
    gcd, x1, y1 = extended_gcd(b % a, a)
    x = y1 - (b // a) * x1
    y = x1
    return gcd, x, y

def mod_inverse(a, m):
    """Нахождение обратного элемента по модулю"""
    gcd, x, _ = extended_gcd(a, m)
    if gcd != 1:
        raise ValueError("Обратный элемент не существует")
    return x % m

In [138]:
class RSA:
    def __init__(
        self,
        alphabet_length: int = 2000,
        open_key: int | None = None,
        secret_key: int | None = None,
        n: int | None = None,
        bits: int = 32,
    ):
        
        self.alphabet_length = alphabet_length

        """Инициализация RSA с генерацией ключей"""
        # Генерируем два простых числа
        if open_key:
            self.open_key = open_key
            self.n = n
        if secret_key:
            self.secret_key = secret_key
            self.n = n

        if open_key is None and secret_key is None:
            self.p = generate_prime(bits // 2)
            self.q = generate_prime(bits // 2)
            self.n = self.p * self.q
            # Вычисляем функцию Эйлера
            self.phi = (self.p - 1) * (self.q - 1)
            self.open_key = 65537
            while math.gcd(self.open_key, self.phi) != 1:
                self.open_key += 2
            self.secret_key = mod_inverse(self.open_key, self.phi)

        self.n_bit_length = self.n.bit_length()


    def encrypt_block(self, block: int):
        """Шифрование сообщения"""

        if block >= self.n:
            raise ValueError("Сообщение слишком большое для шифрования")
        
        return pow(block, self.open_key, self.n)
    
    def decrypt_block(self, cipherblock: int):
        """Дешифрование сообщения"""
        block = pow(cipherblock, self.secret_key, self.n)
        return block
    
    
    def numbers_to_blocks(self, numbers: list[int]) -> list[int]:
        blocks = []
        current_block = new_block = 0
        for num in numbers:
            new_block = (new_block << self.alphabet_length.bit_length())
            new_block |= num
            if new_block.bit_length() > self.n_bit_length:
                blocks.append(current_block)
                current_block = new_block = num
                continue
            current_block = new_block
        else:
            if current_block:
                blocks.append(current_block)
        return blocks[::-1]

    def blocks_to_numbers(self, blocks: list[int]) -> list[int]:
        numbers: list[int] = []
        mask = (1 << self.alphabet_length.bit_length()) - 1
        for block in blocks:
            while True:
                numbers.append(block & mask)
                block >>= self.alphabet_length.bit_length()
                if block <= 0:
                    break
        return numbers[::-1]
    

    def encrypt(self, text: str) -> Any:
        numbers = Coder.encode(text)
        if max(numbers) >= self.alphabet_length:
            raise Exception(f"TOO LOW ALPHABET LENGTH (max alph {max(numbers)=} symbol {chr(max(numbers))})")
        blocks = self.numbers_to_blocks(numbers)

        for i in range(len(blocks)):
            blocks[i] = self.encrypt_block(blocks[i])
        cipher = Coder.encode_blocks(blocks)
        return cipher

    def decrypt(self, cipher: Any) -> str:
        blocks = Coder.decode_blocks(cipher)
        for i in range(len(blocks)):
            blocks[i] = self.decrypt_block(blocks[i])
        numbers = self.blocks_to_numbers(blocks)
        return Coder.decode(numbers)


In [139]:
rsa = RSA(alphabet_length=2000)
print(f"{rsa.alphabet_length=}\n{rsa.open_key=}\n{rsa.secret_key=}\n{rsa.n=}")

rsa.alphabet_length=2000
rsa.open_key=65537
rsa.secret_key=773585549
rsa.n=1915335241


In [140]:
t = "ОЧЕНЬ хорошее сообщение, которому всё равно на alphabet и 123)"
encrypted = rsa.encrypt(t)
decrypted = rsa.decrypt(encrypted)
print(f"{t=} \n{encrypted=} \n{decrypted=}")

t='ОЧЕНЬ хорошее сообщение, которому всё равно на alphabet и 123)' 
encrypted='cipher_rsa.txt' 
decrypted='ОЧЕНЬ хорошее сообщение, которому всё равно на alphabet и 123)'


In [142]:
alphabet_length = input("длина алфавита: ")
open_key = input("открытый ключ: ")
secret_key = input("секретный ключ: ")
n = input("n: ")

rsa = RSA(
    alphabet_length=(int(alphabet_length) if alphabet_length else None),
    open_key=(int(open_key) if open_key else None),
    secret_key=(int(secret_key) if secret_key else None),
    n=(int(n) if n else None)
)


text = input("введите текст для шифрования: ")
if text:
    encrypted = rsa.encrypt(t)
    print(f"{text=} \n{encrypted=} \n")

cipher = input("введите шифр для расшифрования: ")
if cipher:
    decrypted = rsa.decrypt(encrypted)
    print(f"{cipher=} \n{decrypted=} \n")

cipher='cipher_rsa.txt' 
decrypted='ОЧЕНЬ хорошее сообщение, которому всё равно на alphabet и 123)' 

